## Cross in Tray 

In [20]:
using Pkg
Pkg.activate("../../.")
using Globtim
using DynamicPolynomials, DataFrames
using ProgressLogging

  Activating project at `~/Globtim.jl`


In [21]:
# Constants and Parameters
const n, a, b = 2, 10, 1
const scale_factor = a / b   # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = 0.5, 1 / 10  # Sampling parameters
const tol_l2 = 5e-2            # Define the tolerance for the L2-norm
f = CrossInTray # Objective function

CrossInTray (generic function with 1 method)

### Questions to fix:
 - [ ]: Why are we increasing the degree of the approximant and why are we using such a large number of points? The constructor only specifies a few. 
 - [x]: Filter critical points.

In [29]:
d = 8 # Initial Degree 
SMPL = 10 # Number of samples
center = [0.0, 0.0]
TR = test_input(f,
    dim=n,
    center=[0.0, 0.0],
    GN=SMPL,
    sample_range=scale_factor, 
    tolerance=tol_l2,
)
pol_cheb = Constructor(TR, d, basis=:chebyshev)
pol_lege = Constructor(TR, d, basis=:legendre);

attained the desired L2-norm: 0.008027181427042616
Degree :8 
current L2-norm: 0.008027181427042616
Number of samples: 201
attained the desired L2-norm: 0.008657898514056787
Degree :8 
current L2-norm: 0.008657898514056787
Number of samples: 201


In [30]:
@polyvar(x[1:n]) # Define polynomial ring 

(Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}[x₁, x₂],)

In [31]:
real_pts_cheb = solve_polynomial_system(x, TR.dim, pol_cheb.degree, pol_cheb.coeffs; basis=:chebyshev, bigint=true)
real_pts_lege = solve_polynomial_system(x, TR.dim, pol_lege.degree, pol_lege.coeffs; basis=:legendre, bigint=true)
# Usage example:
df_cheb = process_critical_points(real_pts_cheb, f, scale_factor)
df_lege = process_critical_points(real_pts_lege, f, scale_factor)

Tracking 49 paths...  37%|███████████                   |  ETA: 0:00:01



Tracking 49 paths...  55%|████████████████▌             |  ETA: 0:00:01



Tracking 49 paths...  73%|██████████████████████        |  ETA: 0:00:00



Tracking 49 paths...  86%|█████████████████████████▊    |  ETA: 0:00:00



Tracking 49 paths... 100%|██████████████████████████████| Time: 0:00:01
  # paths tracked:                  49
  # non-singular solutions (real):  49 (25)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         49 (25)


Row,x1,x2,z
,Float64,Float64,Float64
1,-1.06198e-15,1.36707e-14,-0.0287829


In [32]:
using GLMakie

# Extract coordinates and function values
# Change the coordinates to uniform grid
coords = pol_cheb.scale_factor * pol_lege.grid
z_coords = pol_lege.z

if size(coords)[2] == 2  # Plot if the dimensions are 2
    fig = Figure(size=(800, 600))
    ax = Axis3(fig[1, 1], title="Cross in Tray",
        xlabel="X-axis", ylabel="Y-axis", zlabel="Z-axis")

    # Scale the domain
    scale_factor = pol_cheb.scale_factor  # Using the scale factor from your polynomial

    # Define threshold for switching between scatter and surface
    point_threshold = 1000  # Adjust this value based on your needs
    sample_fraction = 0.2   # Fraction of points to use if exceeding threshold

    if length(z_coords) > point_threshold
        # Create a regular grid for surface plotting
        x_unique = sort(unique(coords[:, 1]))
        y_unique = sort(unique(coords[:, 2]))

        # Determine grid dimensions
        nx = length(x_unique)
        ny = length(y_unique)

        # Reshape data into a grid
        z_grid = reshape(z_coords, nx, ny)

        # Create surface plot with scaled domain
        surface!(ax, scale_factor .* x_unique, scale_factor .* y_unique, z_grid,
            colormap=:viridis,
            transparency=true,
            alpha=0.8)

        # Sample a fraction of points for scatter overlay
        n_points = length(z_coords)
        sample_indices = rand(1:n_points, Int(floor(n_points * sample_fraction)))

        # Plot sampled scatter points with scaled domain
        scatter!(ax, scale_factor .* coords[sample_indices, 1],
            scale_factor .* coords[sample_indices, 2],
            z_coords[sample_indices],
            markersize=2,
            color=:black,
            label="Sampled Data Points")
    else
        # Original scatter plot for smaller datasets with scaled domain
        scatter!(ax, scale_factor .* coords[:, 1],
            scale_factor .* coords[:, 2],
            z_coords,
            markersize=2,
            color=:black,
            label="Sampled Data Core")

        scatter!(ax, scale_factor .* coords[:, 1],
            scale_factor .* coords[:, 2],
            z_coords,
            markersize=4,
            color=z_coords,
            colormap=:viridis,
            label="Sampled Data Halo")
    end

    # Plot the critical points with scaled domain
    scatter!(ax, scale_factor .* df_cheb.x1,
        scale_factor .* df_cheb.x2,
        df_cheb.z,
        markersize=10,
        color=:orange,
        label="Chebyshev approximant critical points")

    scatter!(ax, scale_factor .* df_lege.x1,
        scale_factor .* df_lege.x2,
        df_lege.z,
        markersize=10,
        color=:yellow,
        label="Legendre approximant critical points")

    display(fig)
end

GLMakie.Screen(...)

In [33]:
# GLMakie.closeall()